# Sensors and Board to Board Communication

This session will cover reading data from the Grove TH02 Temperature and Humidity Sensor as well as transmitting data from one board to another over the internet.

## Part 1: Setup

### 1.1 Initialize Overlay
Our PYNQ Board has a Field Programmable Gate Array (FPGA) on it that must be programmed before we begin using the board. We apply designs called overlays that we can design however we want. For the first part of this tutorial, we are going to use the pre-built base overlay.

In [ ]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

### 1.2 Import Temperature Sensor Libraries
Here we tell the board what libraries we want to use in our project.

In [ ]:
from pynq.lib.arduino import Grove_TH02
from pynq.lib.arduino import ARDUINO_GROVE_I2C

## Part 2: Reading Temperature and Humidity

### 2.1 Plugging in your TH02 Sensor

Plug your PMOD to Grove adaptor (shown in picture below) into one of your PMOD connectors on your PYNQ board. Then plug you TH02 humidity sensor into the one of the four white Grove adaptor ports

![grove_adaptor](https://cdn10.bigcommerce.com/s-7gavg/products/531/images/4194/PYNQ_Shield_-_Oblique_-_600__81639.1473444236.1280.1280.png?c=2)

### 2.2 Initialize the Temperature Sensor

We create an variable called "th02" that represents the sensor. From this object we can call functions that retrieve data from the sensor.

In [ ]:
th02 = Grove_TH02(base.ARDUINO,ARDUINO_GROVE_I2C)

### 2.3 Read the Temperature and Humidity
Calling the function read() returns a tuple (two values separated by a comma). The first value is temperature and the second value is humidity.

In [ ]:
temp_c, humidity = th02.read()
print('The temperature is {} degrees celsius and the relative humidity is {} %'.format(temp_c, humidity))

### 2.4 Convert to Fahrenheit

Converting to Fahrenheit is done by the following equation: 

fahrenheit = celsius * (9/5) + 32

In [ ]:
temp_f = temp_c * (9/5) + 32
print(f'The temperature is {temp_f} F')

### 2.5 Averaging the Temperature

Both computers and the world aren't always perfect. This means that there can be some variation in values that we see. Averaging is a technique that can help up try to hone in on the "correct" answer that we are looking for. 

First, let's read a number of values from the temperature sensor.

In [ ]:
# create temperatures as an empty array
temps = []
# read 5 temperature values and store then in the "temps" array
for i in range(5):
    # read temp in C
    temp_c, _ = th02.read()
    # convert to F
    temp_f = temp_c * (9/5) + 32
    # append temp_f to temps
    temps.append(temp_f)
print(temps)

Now that we have all 5 temperature values stored in an array, let's average them! To average the values, we want to sum them up and then divide the total number of values.

In [ ]:
average_sum = 0
# sum up all the values
for temp in temps:
    average_sum = average_sum + temp
print(f"average_sum: {average_sum}")
average = average_sum / len(temps)
print(f"average: {average}")

## Part 3: Board to Board Communication

In many situations, sensors will need to be a distance away from where the data is needed. For instance, you may want to know the temperature at different locations around a city and be able to monitor this from one central location. To do this, we can send data over the internet. You can also send many types of data beyond this sensor such as text.

In [ ]:
!pip3 install getmac
!pip3 install ./pynq-p2p

In [ ]:
import pynqp2p

### 3.1 Register with the central server
In order to ensure not just anyone can access our data, we use a secret passcode to verify our identity and register with the server that will store our data.

In [ ]:
ip_address = '[IP address of the server goes here]'
key = '[secret passcode goes here]'
pynqp2p.register(ip_address, key)

### 3.2 Make sure we are able to communicate with the server
The function ping() simply sends a message to our server and the server sends a message back saying it heard us.

In [ ]:
pynqp2p.ping()

### 3.3 Get a unique ID from your Board
Your board has a unique ID number called a MAC address. The function get_id() returns a unique number based on this MAC address that you can share with people you want to be able to send you messages. Make sure that the way you share this is secure (such as email) so that you don't have people spamming you with unwanted information.

In [ ]:
myid = pynqp2p.get_id()
print(f"Your ID is {myid}")

### 3.4 Send a message to a board with a particular ID
The function send() sends a message to the board with the particular ID number you pass it.

In [ ]:
recipient_id = 'put your teammates ID here'
pynqp2p.send(recipient_id, "Hello!")

### 3.5 Receive Messages
The function receive() asks the server to give you the most recent message sent to you. Afterwards, it deletes the message (which means you can only receive it once). The function receive_all() retrieves all the messages in the server at once and returns them as a list.

In [ ]:
pynqp2p.receive()

### 3.6 Sending Multiple Messages

Sometimes it is helpful to send multiple messages in a row to another PYNQ board. A really simple way to do this is to run the send(id) command multiple times;

In [ ]:
pynqp2p.send(myid, "Hello 0")
pynqp2p.send(myid, "Hello 1")
pynqp2p.send(myid, "Hello 2")
pynqp2p.send(myid, "Hello 3")
pynqp2p.send(myid, "Hello 4")

Now in order to recieve these messages, we'll have to call the recieve() function a few times.

In [ ]:
print(pynqp2p.receive())
print(pynqp2p.receive())
print(pynqp2p.receive())
print(pynqp2p.receive())
print(pynqp2p.receive())

This seems tedious, there must be a better way! If you remember earlier, we used a for loop to read multiple temperatures from the temperature sensor and put them in an array. We can use the same technique here.

In [ ]:
for i in range(5):
    pynqp2p.send(myid, "Hello " + str(i))

Now we will use another for loop to retrieve all the messages we just sent. 

In [ ]:
for i in range(5):
    print(pynqp2p.receive())

As you can see, this is a lot less tedious and required a lot less code to achieve the same goal. The only problem here is that we can only access the retrieved messages one at a time. pynqp2p containes a really useful function called recieve_all() that helps us here.

In [ ]:
# first, let's send ourselves 5 messages
for i in range(5):
    pynqp2p.send(myid, "Hello " + str(i))

In [ ]:
# now, let's see what the recieve_all() functions does for us
print(pynqp2p.receive_all())

Now you've learned the basics of pynqp2p! Let's try something new. Can you figure out how to send some of the temperature data from earlier? Can you send your Celcius measurement to someone else to be converted into Fahrenheit? Can you send your temperature measurement to someone else and have them respond with weather it's larger or smaller?

The possibilites are endless!